In [1]:
##################################################################################
##### Define all parameters for model tuning
##################################################################################

n_fold = 10
expName = "PSI_Site_DLNN_custom"
outPath = "Results"
foldName = "folds.pickle"

# modelNames = ["DLNN_3", "DLNN_5"]

epochs = 50
batch_size = 16
shuffle = False
seed = None


input_data_folder = "Data\\Psi_Site_Chen"

In [2]:
import os 
from Bio import SeqIO
import pickle
import numpy as np
import pandas as pd

import tensorflow as tf

from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_curve, auc, accuracy_score, precision_score, confusion_matrix
from sklearn.metrics import roc_auc_score

import math

In [3]:
# print(tf.test.is_gpu_available(cuda_only=True))
# physical_devices = tf.config.experimental.list_physical_devices('GPU')
physical_devices = tf.config.list_physical_devices('GPU')
print(physical_devices)
tf.config.experimental.set_memory_growth(physical_devices[0], True)

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [4]:
##################################################################################
##### define all CUSTOM functions
##################################################################################

def one_hot_encode_dna(sequence):
    
    seq_encoded = np.zeros((len(sequence),4))
    dict_nuc = {
        "A": 0,
        "C": 1,
        "G": 2,
        "T":3
    }
    i = 0
    
    for single_character in sequence:
        if(single_character.upper() in dict_nuc.keys()):
            seq_encoded[i][dict_nuc[single_character.upper()]] = 1
            i = i+1
        else:
            return []
    
    return seq_encoded

def one_hot_encode_rna(sequence):
    
    seq_encoded = np.zeros((len(sequence),4))
    dict_nuc = {
        "A": 0,
        "C": 1,
        "G": 2,
        "U":3
    }
    i = 0
    
    for single_character in sequence:
        if(single_character.upper() in dict_nuc.keys()):
            seq_encoded[i][dict_nuc[single_character.upper()]] = 1
            i = i+1
        else:
            return []
    
    return seq_encoded

In [5]:
##################################################################################
##### define evaluator functions
##################################################################################

## Build the K-fold from dataset
def build_kfold(features, labels, k=10, shuffle=False, seed=None):
    
    skf = StratifiedKFold(n_splits=k, shuffle=shuffle, random_state=seed)
    kfoldList = []
    for train_index, test_index in skf.split(features, labels):
        X_train, X_test = features[train_index], features[test_index]
        y_train, y_test = labels[train_index], labels[test_index]
        kfoldList.append({
            "X_train": X_train,
            "X_test": X_test,
            "y_train":y_train,
            "y_test":y_test
        })
    return kfoldList

def pred2label(y_pred):
    y_pred = np.round(np.clip(y_pred, 0, 1))
    return y_pred

In [6]:
# ##################################################################################
# ##### Function to customize the DLNN architecture with parameters
# ##################################################################################

# def DLNN_custom_1(input_shape = (21,4),
#                 conv_filters_per_layer = 10, max_kernel_length = 5, conv_strides = 1, ## 1st Convolutional layer parameters
#                 max_pool_width = 2, max_pool_stride = 2, ## 1st Maxpool layer parameters
#                 lstm_decode_units = 100, ## LSTM layer parameters
#                 dense_decode_units = 100, ## Dense layer parameters
#                 prob = 0.5, learn_rate = 0.0005, loss = 'binary_crossentropy', 
#                 metrics = None):
    
#     input1 = tf.keras.layers.Input(shape=input_shape)

#     ## LSTM Path

#     x1 = tf.keras.layers.LSTM(lstm_decode_units, return_sequences = True)(input1)
# #     x1 = tf.keras.layers.Dropout(prob)(x1)
    
#     x1 = tf.keras.layers.Flatten()(x1)

#     ## Conv Path

#     x2 = tf.keras.layers.Conv1D(conv_filters_per_layer, max_kernel_length, strides = conv_strides)(input1)
#     x2 = tf.keras.layers.Activation('relu')(x2)
# #     x2 = tf.keras.layers.MaxPooling1D(pool_size = max_pool_width, strides = max_pool_stride)(x2)
# #     x2 = tf.keras.layers.Dropout(prob)(x2)
    
#     x2 = tf.keras.layers.Flatten()(x2)

#     ## Fully connected Layers

#     y = tf.keras.layers.Concatenate(1)([x1,x2])
    
#     y1 = tf.keras.layers.Dense(dense_decode_units)(y)
# #     y1 = tf.keras.layers.Dropout(prob)(y1)
    
#     y1 = tf.keras.layers.Dense(dense_decode_units/2)(y1)
# #     y1 = tf.keras.layers.Dropout(prob)(y1)
    
#     y1 = tf.keras.layers.Dense(1, activation = 'sigmoid')(y1)

#     ## Generate Model from input and output
#     model = tf.keras.models.Model(inputs=[input1], outputs=y1)
    
#     ## Compile model
#     if(metrics != None):
#         model.compile(optimizer = tf.keras.optimizers.Adam(lr=learn_rate), loss = loss, metrics = metrics)
#     else:
#         model.compile(optimizer = tf.keras.optimizers.Adam(lr=learn_rate), loss = loss)

#     return model

In [7]:
# ##################################################################################
# ##### Function to customize the DLNN architecture with parameters
# ##################################################################################

# def DLNN_custom_2(input_shape = (21,4),
#                 conv_filters_per_layer = 30, max_kernel_length = 11, conv_strides = 1, ## 1st Convolutional layer parameters
#                 max_pool_width = 2, max_pool_stride = 2, ## 1st Maxpool layer parameters
#                 lstm_decode_units = 100, ## LSTM layer parameters
#                 dense_decode_units = 200, ## Dense layer parameters
#                 prob = 0.5, learn_rate = 0.0005, loss = 'binary_crossentropy', 
#                 metrics = None):
    
#     assert max_kernel_length >= 2
    
#     input1 = tf.keras.layers.Input(shape=input_shape)

#     ## LSTM Path

#     x1 = tf.keras.layers.LSTM(lstm_decode_units, return_sequences = True)(input1)
# #     x1 = tf.keras.layers.Dropout(prob)(x1)
#     x1 = tf.keras.layers.Flatten()(x1)

#     ## Multipath Conv
    
#     x2 = []
#     for kernel_length in range(2,max_kernel_length):
        

#         xx = tf.keras.layers.Conv1D(conv_filters_per_layer, kernel_length, strides = conv_strides)(input1)
# #         xx = tf.keras.layers.Activation('relu')(xx)
#         xx = tf.keras.layers.Flatten()(xx)
        
#         x2.append(xx)
# #     x2 = tf.keras.layers.MaxPooling1D(pool_size = max_pool_width, strides = max_pool_stride)(x2)
# #     x2 = tf.keras.layers.Dropout(prob)(x2)

#     ## Fully connected Layers

#     y = tf.keras.layers.Concatenate(1)([x1]+x2)
    
#     y1 = tf.keras.layers.Dense(dense_decode_units)(y)
#     y1 = tf.keras.layers.Dense(dense_decode_units/2)(y1)
#     y1 = tf.keras.layers.Dense(1, activation = 'sigmoid')(y1)
# #     y1 = tf.keras.layers.Dropout(prob)(y1)

# #     y1 = tf.keras.layers.Dense(dense_decode_units)(y1)
    
# #     y1 = tf.keras.layers.Dense(dense_decode_units/2)(y1)
# #     y1 = tf.keras.layers.Dropout(prob)(y1)
    
# #     y1 = tf.keras.layers.Dense(1, activation = 'sigmoid')(y1)
# #     y1 = tf.keras.layers.Dense(1, activation = 'sigmoid')(y1)

#     ## Generate Model from input and output
#     model = tf.keras.models.Model(inputs=[input1], outputs=y1)
    
#     ## Compile model
#     if(metrics != None):
#         model.compile(optimizer = tf.keras.optimizers.Adam(lr=learn_rate), loss = loss, metrics = metrics)
#     else:
#         model.compile(optimizer = tf.keras.optimizers.Adam(lr=learn_rate), loss = loss)

#     return model

In [8]:
# ##################################################################################
# ##### Function to customize the DLNN architecture with parameters
# ##################################################################################

# def DLNN_custom_3(input_shape = (21,4),
#                 conv_filters_per_layer = 10, max_kernel_length = 11, conv_strides = 1, ## 1st Convolutional layer parameters
#                 max_pool_width = 2, max_pool_stride = 2, ## 1st Maxpool layer parameters
#                 lstm_decode_units = 50, ## LSTM layer parameters
#                 dense_decode_units = 100, ## Dense layer parameters
#                 prob = 0.25, learn_rate = 0.001, loss = 'binary_crossentropy', 
#                 metrics = None):
    
#     assert max_kernel_length >= 2
    
#     input1 = tf.keras.layers.Input(shape=input_shape)

#     ##### LSTM Path
#     #########################

# #     x1 = tf.keras.layers.LSTM(lstm_decode_units, return_sequences = True)(input1)
# #     x1 = tf.keras.layers.LSTM(1, return_sequences = True)(x1)
# #     x1 = tf.keras.layers.Dropout(prob)(x1)
    
#     x1 = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(lstm_decode_units, return_sequences=True))(input1)
    
# #     model.add(Bidirectional(tf.keras.layers.LSTM(21)))
    
#     x1 = tf.keras.layers.Flatten()(x1)
#     x1 = tf.keras.layers.Dropout(prob)(x1)

#     ##### Multipath Conv
#     #########################
    
#     x2 = []
#     for kernel_length in range(2,max_kernel_length):
        

#         xx = tf.keras.layers.Conv1D(conv_filters_per_layer, kernel_length, strides = conv_strides)(input1)
# #         xx = tf.keras.layers.Activation('relu')(xx)
#         xx = tf.keras.layers.Flatten()(xx)
#         xx = tf.keras.layers.Dropout(prob)(xx)
        
#         x2.append(xx)
# #     x2 = tf.keras.layers.MaxPooling1D(pool_size = max_pool_width, strides = max_pool_stride)(x2)
# #     x2 = tf.keras.layers.Dropout(prob)(x2)

#     ##### Fully connected Layers
#     #########################

#     y = tf.keras.layers.Concatenate(1)([x1]+x2)
    
#     y1 = tf.keras.layers.Dense(dense_decode_units)(y)
#     y1 = tf.keras.layers.Dropout(prob)(y1)
#     y1 = tf.keras.layers.Dense(dense_decode_units/2)(y1)
#     x1 = tf.keras.layers.Dropout(prob)(x1)
#     y1 = tf.keras.layers.Dense(1, activation = 'sigmoid')(y1)
# #     y1 = tf.keras.layers.Dropout(prob)(y1)

# #     y1 = tf.keras.layers.Dense(dense_decode_units)(y1)
    
# #     y1 = tf.keras.layers.Dense(dense_decode_units/2)(y1)
# #     y1 = tf.keras.layers.Dropout(prob)(y1)
    
# #     y1 = tf.keras.layers.Dense(1, activation = 'sigmoid')(y1)
# #     y1 = tf.keras.layers.Dense(1, activation = 'sigmoid')(y1)

#     ##### Generate Model from input and output
#     #########################
    
#     model = tf.keras.models.Model(inputs=[input1], outputs=y1)
    
#     ## Compile model
#     if(metrics != None):
#         model.compile(optimizer = tf.keras.optimizers.Adam(lr=learn_rate), loss = loss, metrics = metrics)
#     else:
#         model.compile(optimizer = tf.keras.optimizers.Adam(lr=learn_rate), loss = loss)

#     return model

In [9]:
# ##################################################################################
# ##### Function to customize the DLNN architecture with parameters
# ##################################################################################

# def DLNN_custom_4(input_shape = (21,4),
#                   conv_filters_per_layer = 5, max_kernel_length = 11, conv_strides = 1, ## 1st Convolutional layer parameters
#                   max_pool_width = 2, max_pool_stride = 2, ## 1st Maxpool layer parameters
#                   rnn_decode_units = 25, ## LSTM layer parameters
#                   dense_decode_units = 100, ## Dense layer parameters
#                   prob = 0.5, learn_rate = 0.001, loss = 'binary_crossentropy', 
#                   metrics = None):
    
#     # beta = 0.001
    
#     assert max_kernel_length >= 2
    
#     input1 = tf.keras.layers.Input(shape=input_shape)
    
#     #########################
#     ##### LSTM Path
#     #########################
    
#     x1 = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(rnn_decode_units, 
#                                                             return_sequences=True, 
#                                                             return_state=False))(input1)
#     x1 = tf.keras.layers.Dropout(prob)(x1)
#     x1 = tf.keras.layers.Flatten()(x1)
    
#     #########################
#     ##### Multipath Conv
#     #########################
    
#     x2 = []
#     for kernel_length in range(2, max_kernel_length):
        
#         xx = tf.keras.layers.Conv1D(conv_filters_per_layer, kernel_length, strides = conv_strides)(input1)
#         xx = tf.keras.layers.Activation('relu')(xx)
#         xx = tf.keras.layers.MaxPool1D(pool_size = max_pool_width, strides = max_pool_stride)(xx)
#         xx = tf.keras.layers.Dropout(prob)(xx)
#         xx = tf.keras.layers.Flatten()(xx)
        
#         x2.append(xx)
        
#     #########################
#     ##### Fully connected Layers
#     #########################

#     y = tf.keras.layers.Concatenate(1)([x1]+x2)
    
#     y1 = tf.keras.layers.Dense(dense_decode_units)(y)
#     y1 = tf.keras.layers.Activation('relu')(y1)
#     y1 = tf.keras.layers.Dropout(prob)(y1)
    
#     y1 = tf.keras.layers.Dense(1, activation = 'sigmoid')(y1)
    
#     #########################
#     ##### Generate Model from input and output
#     #########################
    
#     model = tf.keras.models.Model(inputs=[input1], outputs=y1)
    
#     ## Compile model
#     if(metrics != None):
#         model.compile(optimizer = tf.keras.optimizers.Adam(lr=learn_rate), loss = loss, metrics = metrics)
#     else:
#         model.compile(optimizer = tf.keras.optimizers.Adam(lr=learn_rate), loss = loss)

#     return model

In [10]:
##################################################################################
##### Function to customize the DLNN architecture with parameters
##################################################################################

def DLNN_custom_5(input_shape = (21,4),
                  conv_filters_per_layer = 10, max_kernel_length = 11, conv_strides = 1, ## 1st Convolutional layer parameters
                  max_pool_width = 2, max_pool_stride = 2, ## 1st Maxpool layer parameters
                  rnn_decode_units = 10, ## LSTM layer parameters
                  dense_decode_units = 100, ## Dense layer parameters
                  prob = 0.5, learn_rate = 0.001, loss = 'binary_crossentropy', 
                  metrics = None):
    
    beta = 0.01
    
    assert max_kernel_length >= 2
    
    input1 = tf.keras.layers.Input(shape=input_shape)
    
    #########################
    ##### LSTM Path
    #########################
    
    x1 = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(rnn_decode_units*5, 
                                                            return_sequences=True, 
                                                            return_state=False))(input1)
    x1 = tf.keras.layers.Dropout(prob)(x1)
    x1 = tf.keras.layers.Flatten()(x1)
    
    #########################
    ##### Multipath Conv-LSTM
    #########################
    
    x2 = []
    for kernel_length in range(2, max_kernel_length):
        
#         xx = tf.keras.layers.Conv1D(conv_filters_per_layer, kernel_length, 
#                                     strides = conv_strides, kernel_regularizer = tf.keras.regularizers.l2(beta))(input1)
        xx = tf.keras.layers.Conv1D(conv_filters_per_layer, kernel_length, 
                                    strides = conv_strides)(input1)
        
        xxc = tf.keras.layers.Activation('relu')(xx)
        xxc = tf.keras.layers.MaxPool1D(pool_size = max_pool_width, strides = max_pool_stride)(xxc)
        xxc = tf.keras.layers.Dropout(prob)(xxc)
        xxc = tf.keras.layers.Flatten()(xxc)
        x2.append(xxc)
        
        xxl = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(int((input_shape[0]-kernel_length)/2), 
                                                                 return_sequences=True, 
                                                                 return_state=False))(xx)
        xxl = tf.keras.layers.Dropout(prob)(xxl)
        xxl = tf.keras.layers.Flatten()(xxl)
        x2.append(xxl)
        
    #########################
    ##### Fully connected Layers
    #########################

    y = tf.keras.layers.Concatenate(1)([x1]+x2)
    
    y1 = tf.keras.layers.Dense(dense_decode_units)(y)
    y1 = tf.keras.layers.Activation('relu')(y1)
    y1 = tf.keras.layers.Dropout(prob)(y1)
    
    y1 = tf.keras.layers.Dense(1, activation = 'sigmoid')(y1)
    
    #########################
    ##### Generate Model from input and output
    #########################
    
    model = tf.keras.models.Model(inputs=[input1], outputs=y1)
    
    ## Compile model
    if(metrics != None):
        model.compile(optimizer = tf.keras.optimizers.Adam(lr=learn_rate), loss = loss, metrics = metrics)
    else:
        model.compile(optimizer = tf.keras.optimizers.Adam(lr=learn_rate), loss = loss)

    return model

In [11]:
DLNN_custom_5().summary()


KeyboardInterrupt



In [ ]:
# tf.keras.utils.plot_model(DLNN_custom_5())

In [ ]:
##################################################################################
##### For each input file, train model and generate different outputs in a structured folder
##################################################################################

## create the evaluation data structure for all iterations
evaluations = {
    "Model" : [],
    "Kernel_Length" : [],
    "Dataset" : [],
    "Fold" : [],
    "Train_Test" : [],
    "Accuracy" : [],
    "Precision": [],
    "TPR": [],
    "FPR": [],
    "TPR_FPR_Thresholds": [],
    "AUC": [],
    "Sensitivity": [],
    "Specificity": [],
    "MCC":[]
}

for root, dirs, files in os.walk(input_data_folder):
    for file in files:
        
        input_data_file = os.path.join(root, file)
        
        current_dataset_variety = input_data_file.split("\\")[-1].split(".")[0]
        
        openFile = open(input_data_file)
        fastaSequences = SeqIO.parse(openFile, "fasta")
        
        ##################################################################################
        ##### extract data from the current fasta file
        ##################################################################################

        positive_List = []
        negative_List = []
        positive_onehotencoded_List = []
        negative_onehotencoded_List = []

        for fasta in fastaSequences: 
            name, sequence = fasta.id, str(fasta.seq)
            if "P" in name:
                positive_List.append(sequence)
                aus_seq = one_hot_encode_rna(sequence)
                if(len(aus_seq) != 0):
                    positive_onehotencoded_List.append(aus_seq)
            elif "N" in name:
                negative_List.append(sequence)
                aus_seq = one_hot_encode_rna(sequence)
                if(len(aus_seq) != 0):
                    negative_onehotencoded_List.append(aus_seq)

        openFile.close()

        print("\n======================================================================")
        print("\nFile: "+os.path.join(root, file))
        print("Positive: "+str(len(positive_onehotencoded_List)))
        print("Negative: "+str(len(negative_onehotencoded_List)))
        
        ##################################################################################
        ##### Generate Folds from dataset, and store to file
        ##################################################################################

        ## create the features and labels datasets for the training
        input_size = (len(positive_onehotencoded_List[1]), 4)
        labels = np.concatenate((np.ones((len(positive_onehotencoded_List), 1), dtype=np.float32), np.zeros((len(negative_onehotencoded_List), 1), dtype=np.float32)), axis=0)
        features = np.concatenate((positive_onehotencoded_List,negative_onehotencoded_List), 0)

        ## Generate the k-fold dataset
        folds = build_kfold(features, labels, k=n_fold, shuffle=shuffle, seed=seed)

        ## Write the k-fold dataset to file
        foldPath = os.path.join(outPath, expName, current_dataset_variety, "{}fold".format(n_fold))
        if(not os.path.isdir(foldPath)):
            os.makedirs(foldPath)
        pickle.dump(folds, open(os.path.join(foldPath, foldName), "wb"))

        ## Create and set directory to save model
        modelPath = os.path.join(outPath, expName, current_dataset_variety, "{}fold".format(n_fold), "models")
        if(not os.path.isdir(modelPath)):
            os.makedirs(modelPath)
            
        ##################################################################################
        ##### TRAIN and PREDICT for every Fold, using models
        ##################################################################################

        # fold counter
        i = 0

        for fold in folds:

            print("\nTrain/Test model "+current_dataset_variety+" on Fold #"+str(i)+".")

            kernel_length = 3
            ## Generate model using function
#             model = Conv_LSTM_DLNN(input_shape = input_size, conv_filters_per_layer = 50, kernel_length = kernel_length, 
#                                    lstm_decode_units = 50, max_pool_width = 2, max_pool_stride = 2, dense_decode_units = 50,
#                                    learn_rate = 0.0001, prob = 0.5, loss='binary_crossentropy', metrics=None)

            model = DLNN_custom_5(input_shape = input_size)

#             ## Define the model callbacks for early stopping and saving the model. Then train model
            modelCallbacks = [
                tf.keras.callbacks.ModelCheckpoint(os.path.join(modelPath, "{}_bestModel-fold{}.hdf5".format(current_dataset_variety, i)),
                                                   monitor = 'val_loss', verbose = 0, save_best_only = True, 
                                                   save_weights_only = False, mode = 'auto', save_freq = 'epoch'),
                tf.keras.callbacks.EarlyStopping(monitor = 'val_loss', min_delta = 0, patience = 10, verbose = 0, 
                                                 mode = 'auto', baseline = None, restore_best_weights = False)
            ]
            model.fit(x = fold["X_train"], y = fold["y_train"], batch_size = batch_size, epochs = epochs, verbose = 0, 
                      callbacks = modelCallbacks, validation_split=0.2)

#             model.fit(x = fold["X_train"], y = fold["y_train"], batch_size = batch_size, epochs = epochs, verbose = 1, 
#                       validation_split=0.2)

            ##################################################################################
            ##### Prediction and metrics for TRAIN dataset
            ##################################################################################

            y_pred = model.predict(fold["X_train"])
            label_pred = pred2label(y_pred)
            # Compute precision, recall, sensitivity, specifity, mcc
            acc = accuracy_score(fold["y_train"], label_pred)
            prec = precision_score(fold["y_train"],label_pred)

            conf = confusion_matrix(fold["y_train"], label_pred)
            if(conf[0][0]+conf[1][0]):
                sens = float(conf[0][0])/float(conf[0][0]+conf[1][0])
            else:
                sens = 0.0
            if(conf[1][1]+conf[0][1]):
                spec = float(conf[1][1])/float(conf[1][1]+conf[0][1])
            else:
                spec = 0.0
            if((conf[0][0]+conf[0][1])*(conf[0][0]+conf[1][0])*(conf[1][1]+conf[0][1])*(conf[1][1]+conf[1][0])):
                mcc = (float(conf[0][0])*float(conf[1][1]) - float(conf[1][0])*float(conf[0][1]))/math.sqrt((conf[0][0]+conf[0][1])*(conf[0][0]+conf[1][0])*(conf[1][1]+conf[0][1])*(conf[1][1]+conf[1][0]))
            else:
                mcc= 0.0
            fpr, tpr, thresholds = roc_curve(fold["y_train"], y_pred)
            auc = roc_auc_score(fold["y_train"], y_pred)

            evaluations["Model"].append(current_dataset_variety)
            evaluations["Kernel_Length"].append(kernel_length)
            evaluations["Dataset"].append(current_dataset_variety)
            evaluations["Fold"].append(i)
            evaluations["Train_Test"].append("Train")
            evaluations["Accuracy"].append(acc)
            evaluations["Precision"].append(prec)
            evaluations["TPR"].append(tpr)
            evaluations["FPR"].append(fpr)
            evaluations["TPR_FPR_Thresholds"].append(thresholds)
            evaluations["AUC"].append(auc)
            evaluations["Sensitivity"].append(sens)
            evaluations["Specificity"].append(spec)
            evaluations["MCC"].append(mcc)

            ##################################################################################
            ##### Prediction and metrics for TEST dataset
            ##################################################################################

            y_pred = model.predict(fold["X_test"])
            label_pred = pred2label(y_pred)
            # Compute precision, recall, sensitivity, specifity, mcc
            acc = accuracy_score(fold["y_test"], label_pred)
            prec = precision_score(fold["y_test"],label_pred)

            conf = confusion_matrix(fold["y_test"], label_pred)
            if(conf[0][0]+conf[1][0]):
                sens = float(conf[0][0])/float(conf[0][0]+conf[1][0])
            else:
                sens = 0.0
            if(conf[1][1]+conf[0][1]):
                spec = float(conf[1][1])/float(conf[1][1]+conf[0][1])
            else:
                spec = 0.0
            if((conf[0][0]+conf[0][1])*(conf[0][0]+conf[1][0])*(conf[1][1]+conf[0][1])*(conf[1][1]+conf[1][0])):
                mcc = (float(conf[0][0])*float(conf[1][1]) - float(conf[1][0])*float(conf[0][1]))/math.sqrt((conf[0][0]+conf[0][1])*(conf[0][0]+conf[1][0])*(conf[1][1]+conf[0][1])*(conf[1][1]+conf[1][0]))
            else:
                mcc= 0.0
            fpr, tpr, thresholds = roc_curve(fold["y_test"], y_pred)
            auc = roc_auc_score(fold["y_test"], y_pred)

            evaluations["Model"].append(current_dataset_variety)
            evaluations["Kernel_Length"].append(kernel_length)
            evaluations["Dataset"].append(current_dataset_variety)
            evaluations["Fold"].append(i)
            evaluations["Train_Test"].append("Test")
            evaluations["Accuracy"].append(acc)
            evaluations["Precision"].append(prec)
            evaluations["TPR"].append(tpr)
            evaluations["FPR"].append(fpr)
            evaluations["TPR_FPR_Thresholds"].append(thresholds)
            evaluations["AUC"].append(auc)
            evaluations["Sensitivity"].append(sens)
            evaluations["Specificity"].append(spec)
            evaluations["MCC"].append(mcc)

            i = i+1
            del model
            tf.keras.backend.clear_session()

        ##################################################################################
        ##### Dump evaluations to a file
        ##################################################################################

        evalPath = os.path.join(outPath, expName, "_Evaluation_All_Datasets")
        if(not os.path.isdir(evalPath)):
            os.makedirs(evalPath)

        pickle.dump(evaluations,
                    open(os.path.join(evalPath, "{}fold_evaluations.pickle".format(n_fold)), "wb"))

## Visualization of Evaluation

In [ ]:
##################################################################################
##### Add import statement here, to make this next part of code standalone executable
##################################################################################

import os
import pickle
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.ticker import ScalarFormatter, FormatStrFormatter
import numpy as np
import pandas as pd


In [ ]:
##################################################################################
##### Load file and convert to dataframe for easy manipulation
##################################################################################

evalPath = os.path.join(outPath, expName, "_Evaluation_All_Datasets")
if(not os.path.isdir(evalPath)):
    os.makedirs(evalPath)

evaluations = pickle.load(open(os.path.join(evalPath, "{}fold_evaluations.pickle".format(n_fold)), "rb"))

In [ ]:
# evaluations["Model"] = evaluations["Model"][0:20]
# evaluations_df = pd.DataFrame.from_dict(evaluations)

In [ ]:
evaluations_df = pd.DataFrame.from_dict(evaluations)

##################################################################################
##### Group dataset (mean of metrics) by [Dataset, Model, Train_Test] combinations
##################################################################################

evaluations_df_grouped = evaluations_df.groupby(["Dataset", 
                                                 "Model", 
                                                 "Train_Test"]).mean().filter(['Accuracy', 
                                                                               'Precision', 
                                                                               'AUC', 
                                                                               'Sensitivity', 
                                                                               'Specificity', 
                                                                               'MCC'])

# DLNN_3 = evaluations_df_grouped[np.in1d(evaluations_df_grouped.index.get_level_values(1), ['DLNN_3'])]
# DLNN_5 = evaluations_df_grouped[np.in1d(evaluations_df_grouped.index.get_level_values(1), ['DLNN_5'])]

# DLNN_3_Train = DLNN_3[np.in1d(DLNN_3.index.get_level_values(2), ['Train'])]
# DLNN_3_Test = DLNN_3[np.in1d(DLNN_3.index.get_level_values(2), ['Test'])]

# DLNN_5_Train = DLNN_5[np.in1d(DLNN_5.index.get_level_values(2), ['Train'])]
# DLNN_5_Test = DLNN_5[np.in1d(DLNN_5.index.get_level_values(2), ['Test'])]

In [ ]:
evaluations_df_grouped

In [ ]:
# ##################################################################################
# ##### Decide on metric to visualize
# ##################################################################################

# print("Metrics Available : ")
# print(list(evaluations_df_grouped.columns))

#### Select a metric to plot below:

In [ ]:
# metric_to_plot = "Accuracy"

In [ ]:
# ##################################################################################
# ##### Visualize with a multiple Bar chart
# ##################################################################################

# x = np.arange(len(DLNN_3_Train[metric_to_plot]))
# width = 0.15

# fig, ax = plt.subplots(figsize=(17,6))
# rects1 = ax.bar(x - (4*(width/2)), round(DLNN_3_Train[metric_to_plot]*100, 3), width, label='DLNN_3, Train')
# rects2 = ax.bar(x - (1.5*(width/2)), round(DLNN_5_Train[metric_to_plot]*100, 3), width, label='DLNN_5, Train')
# rects3 = ax.bar(x + (1.5*(width/2)), round(DLNN_3_Test[metric_to_plot]*100, 3), width, label='DLNN_3, Test')
# rects4 = ax.bar(x + (4*(width/2)), round(DLNN_5_Test[metric_to_plot]*100, 3), width, label='DLNN_5, Test')

# ## Custom y-axis tick labels
# ax.set_ylabel(metric_to_plot)
# ax.set_ylim([(math.floor(min(evaluations_df_grouped[metric_to_plot])*10)-1)*10, 
#             (math.ceil(max(evaluations_df_grouped[metric_to_plot])*10)+1)*10])
# # ax.set_ylim([80, 105])

# ## Custom x-axis tick labels
# ax.set_xticks(x)
# # ax.set_xticklabels(DLNN_3_Train.index.get_level_values(0))
# # ax.set_xticklabels([m+" - "+str(n) for m,n in 
# #                         zip(DLNN_3_Train.index.get_level_values(0),DLNN_3_Train.index.get_level_values(1))],
# #                   rotation=30)
# ax.set_xticklabels(DLNN_3_Train.index.get_level_values(0))

# ax.set_title(metric_to_plot+' by Dataset, Model, Train/Test')
# ax.legend(loc='upper left')

# def autolabel(rects):
#     for rect in rects:
#         height = rect.get_height()
#         ax.annotate('{}'.format(height),
#                     xy=(rect.get_x() + rect.get_width() / 2, height),
#                     xytext=(0, 3),  # 3 points vertical offset
#                     textcoords="offset points", 
#                     ha='center', va='bottom', rotation=90)

# autolabel(rects1)
# autolabel(rects2)
# autolabel(rects3)
# autolabel(rects4)

# plt.show()

### Store all metrics' plots to file

In [ ]:
# ##################################################################################
# ##### Iteratively generate comparison plot using every metric
# ##################################################################################

# for metric_to_plot in list(evaluations_df_grouped.columns):
    
#     x = np.arange(len(DLNN_3_Train[metric_to_plot]))
#     width = 0.15

#     fig, ax = plt.subplots(figsize=(17,6))
#     rects1 = ax.bar(x - (4*(width/2)), round(DLNN_3_Train[metric_to_plot]*100, 3), width, label='DLNN_3, Train')
#     rects2 = ax.bar(x - (1.5*(width/2)), round(DLNN_5_Train[metric_to_plot]*100, 3), width, label='DLNN_5, Train')
#     rects3 = ax.bar(x + (1.5*(width/2)), round(DLNN_3_Test[metric_to_plot]*100, 3), width, label='DLNN_3, Test')
#     rects4 = ax.bar(x + (4*(width/2)), round(DLNN_5_Test[metric_to_plot]*100, 3), width, label='DLNN_5, Test')

#     ## Custom y-axis tick labels
#     ax.set_ylabel(metric_to_plot)
#     ax.set_ylim([(math.floor(min(evaluations_df_grouped[metric_to_plot])*10)-1)*10, 
#                 (math.ceil(max(evaluations_df_grouped[metric_to_plot])*10)+1)*10])
#     # ax.set_ylim([80, 105])

#     ## Custom x-axis tick labels
#     ax.set_xticks(x)
#     # ax.set_xticklabels(DLNN_3_Train.index.get_level_values(0))
#     # ax.set_xticklabels([m+" - "+str(n) for m,n in 
#     #                         zip(DLNN_3_Train.index.get_level_values(0),DLNN_3_Train.index.get_level_values(1))],
#     #                   rotation=30)
#     ax.set_xticklabels(DLNN_3_Train.index.get_level_values(0))

#     ax.set_title(metric_to_plot+' by Dataset, Model, Train/Test')
#     ax.legend(loc='upper left')

#     def autolabel(rects):
#         for rect in rects:
#             height = rect.get_height()
#             ax.annotate('{}'.format(height),
#                         xy=(rect.get_x() + rect.get_width() / 2, height),
#                         xytext=(0, 3),  # 3 points vertical offset
#                         textcoords="offset points", 
#                         ha='center', va='bottom', rotation=90)

#     autolabel(rects1)
#     autolabel(rects2)
#     autolabel(rects3)
#     autolabel(rects4)
    
#     plt.savefig(os.path.join(evalPath, "{}_DLNN_Comparison".format(metric_to_plot)))
#     plt.close()
    